In [1]:
from whoosh.index import create_in
from whoosh.fields import *

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

schema = Schema(title_ix=TEXT(stored=True), content_ix=TEXT(stored=True), url_ix=TEXT(stored=True))

ix = create_in("indexdir", schema)
writer = ix.writer()

prefix = 'https://vm009.rz.uos.de/crawl/'
start_url = prefix+'index.html'
agenda = [start_url]
visited_recently = []
headers_v = []


while agenda:
    url = agenda.pop()
    if url not in visited_recently:
        visited_recently.append(url)
        r = requests.get(url)

        if r.status_code == 200:
            try:
                soup = BeautifulSoup(r.content, 'html.parser')

                content = soup.get_text()
                writer.add_document(title_ix = soup.title.get_text(strip=True) if soup.title else "No Title", content_ix=soup.get_text(), url_ix=url)
                
                for link in soup.find_all('a'):
                    
                    href = link.get('href')
                    # if the href is a relative link, we need to join it with the base url. If the href is an absolut link the function urljoin will return the href as it is.
                    full_url = urljoin(url, href)

                    # we dont want to scrape other weppages than the ones that start with the prefix
                    if full_url.startswith(prefix):
                            agenda.append(full_url)
            except:
                print(f"Error processing {url}")
writer.commit()


In [2]:
from whoosh.qparser import QueryParser
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.index import open_dir  
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

ix = open_dir("indexdir")
with ix.searcher() as searcher:
    
    query = QueryParser("content_ix", ix.schema).parse("unicorn")
    results = searcher.search(query)
    results.fragmenter.surround = 50
    # print all results
    for r in results:
        print(r["title_ix"])
        print(r.highlights("content_ix", top = 1))

Page 6
bytes so torn,
Lived a quirky platypus and a geeky <b class="match term0">unicorn</b>.
Platypus coded, with fins so deft,
While <b class="match term0">unicorn</b> debugged, with every breath left.

"Did you know," said Plat
Page 4
Page 4


This is Page 4
The <b class="match term0">unicorn</b> is a legendary creature that has been described since
Page 5
This is Page 5
In European literature and art, the <b class="match term0">unicorn</b> has for the last thousand years or so been depicted as
